In [ ]:
import glob
import os

import numpy as np

from pynextsim import NextsimBin

In [ ]:
USER = os.getenv('USER')
NODE_VARS = ['M_VT', 'M_wind', 'M_ocean']
ELEM_VARS = [
    'Damage',
    'Concentration',
    'Thickness',
    'Sigma_0',
    'Sigma_1',
]

def bin2npz(field_file):
    npz_file = field_file.replace('.bin', '.npz')
    n = NextsimBin(field_file)
    n_nodes = n.mesh_info.nodes_x.size
    
    d = {name: n.get_var(name).astype(np.float32) for name in ELEM_VARS}
    for name in NODE_VARS:
        d[f'{name}_x'] = n.get_var(name)[:n_nodes].astype(np.float32)
        d[f'{name}_y'] = n.get_var(name)[n_nodes:].astype(np.float32)

    d['x'] = n.mesh_info.nodes_x
    d['y'] = n.mesh_info.nodes_y
    d['t'] = n.mesh_info.indices
    d['i'] = n.mesh_info.get_var('id')
    np.savez(npz_file, **d)

In [ ]:
idir = f'/data2/{USER}/fran/'
field_files = sorted(glob.glob(f'{idir}/field_2*bin'))
for field_file in field_files:
    bin2npz(field_file)